# match the name to the latitude and longitude
In this section, we are augmenting the original data with external information. 

Mutiple countries:
    Some cities will return multiple matches with the previous function in different countries. 
    You'll have to decide which city to keep based on a heuristic (rule of thumb).

Result:
    It will be helpful to get the countrycode of each headline at this point.
    You should end up with over 600 headlines that have geographic coordinates.
    Compute summary statistics and make basic distribution plots (histograms) to search for anomalies.

Goal:
    Our hypothesis is that there may be groups of headlines in close proximity referring to similar diseases. 

In [17]:
import pandas as pd
import numpy as np
import unidecode
from geonamescache import GeonamesCache
gc = GeonamesCache()

import matplotlib.pyplot as plt
%matplotlib inline #Jupyter “magic” function that enables plt to plot data in the cell where your code is. 

import warning
warnings.simplefilter("ignore")


UsageError: unrecognized arguments: #Jupyter “magic” function that enables plt to plot data in the cell where your code is.


In [69]:
hl = pd.read_csv('E:/Self-Dev/ManningProject-Headlines/data/hl.csv').set_index('Unnamed: 0')
hl[['latitude','longitude','countrycode']] = pd.DataFrame([[np.nan]*3 for i in range(hl.shape[0])])

hl = hl.replace(np.nan, '', regex=True) # cannot use isnan on str

hl.head()

,headline,city,country,latitude,longitude,countrycode
Unnamed: 0,,,,,,
0,Zika Outbreak Hits Miami,Miami,United States,,,
1,Could Zika Reach New York City?,New York City,United States,,,
2,First Case of Zika in Miami Beach,Miami Beach,United States,,,
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil,,,
4,Dallas man comes down with case of Zika,Dallas,,,,


## Match Coordinates

### No City or Country Matches
I choose some of them to analysis, research on the problem why matching fails.

In [70]:
# hl.loc[no_city_info] # no records
no_info = []
for ix in hl.index:
    if len(hl.city[ix]) == 0 and len(hl.country[ix]) == 0:
        no_info.append(ix)
print(hl.loc[no_info].shape)

# problem of multiple names of a city
hl.loc[19]['city'] = 'Cebu City' 
hl.loc[19]['country'] = 'Philippines'

# smaller than a city
hl.loc[48]['country'] = 'Antigua and Barbuda' # part of island

hl.loc[73]['city'] = 'Iloilo' # only a town
hl.loc[73]['country'] = 'Philippines'

hl.loc[156]['city'] = 'New York City' # university/church name
hl.loc[156]['country'] = 'United States'

hl.loc[234]['city'] = 'Chichester'
hl.loc[234]['country'] = 'United Kingdom'

# case sensitivity
hl.loc[63]['city'] = 'Rio de Janeiro' # case sensitivity
hl.loc[63]['country'] = 'Brazil'

hl.loc[139]['city'] = 'McAllen' 
hl.loc[139]['country'] = 'United States'

hl.loc[233]['city'] = 'McLean' 
hl.loc[233]['country'] = 'United States'

(41, 6)


### Add Latitude and Longitude info

In [75]:
countries = gc.get_countries_by_names()
cities = gc.get_cities()

mult_match = dict()
no_match = []
for ix in hl.index:
    if len(hl.city[ix]) > 0:
        city = hl.city[ix]
        match = gc.get_cities_by_name(city)
        if len(match) > 1:
            #print('1',hl.loc[4])
            mult_match[ix] = match
        elif len(match) == 0:
            #print('2',hl.loc[4])
            no_match.append(ix)
        else:
            #print('3',hl)
            match = match[0]
            hl.latitude[ix] = match[list(match.keys())[0]]['latitude']
            hl['longitude'][ix] = match[list(match.keys())[0]]['longitude']
            hl.countrycode[ix] = match[list(match.keys())[0]]['countrycode']
    elif len(hl.country[ix]) > 0:   # if only country info could be found, I use the average coordinates to represent it
        countrycode = countries[hl.country[ix]]['iso']
        hl.countrycode[ix] = countrycode
        all_cities = [city for city in cities.values() if city['countrycode'] == countrycode]
        center_lat = np.mean([city['latitude'] for city in all_cities])
        center_lon = np.mean([city['longitude'] for city in all_cities])
        hl['latitude'][ix] = center_lat
        hl['longitude'][ix] = center_lon

## Analysis on Abnormal Results

### Multiple Match

In [80]:
code_list = gc.get_countries()

In [102]:
for ix in mult_match.keys():
    match = mult_match[ix]
    pop = 0
    lat = 0
    long = 0
    code = 0
    for a in match:
        this = a[list(a.keys())[0]]['population']
        if this > pop:
            pop = this
            lat = a[list(a.keys())[0]]['latitude']
            long = a[list(a.keys())[0]]['longitude']
            code = a[list(a.keys())[0]]['countrycode']
    hl.latitude[ix] = lat
    hl.longitude[ix] = long
    hl.countrycode[ix] = code
    hl.country[ix] = code_list[code]['name']

### No Match
Some because accents;
Louisiana is the only one which cannot be found, I manually attached its coordinates

In [104]:
num = len(no_match)
print(num)
find = []
for ix in no_match:
    city = hl.city[ix]
    for key in cities.keys():
        if unidecode.unidecode(cities[key]['name']) == city:
            hl.latitude[ix] = cities[key]['latitude']
            hl.longitude[ix] = cities[key]['longitude']
            hl.countrycode[ix] = cities[key]['countrycode']
            num -= 1
            find.append(ix)
            continue
print(num)

26
1


In [105]:
hl.loc[[i for i in no_match if i not in find]]

,headline,city,country,latitude,longitude,countrycode
Unnamed: 0,,,,,,
17,Louisiana Zika cases up to 26,Louisiana,United States,30.9843,-91.9623,


In [108]:
hl.latitude[17] = 30.9843
hl.longitude[17] = -91.9623
hl.countrycode[17] = 'US'

In [109]:
hl.to_csv('E:/Self-Dev/ManningProject-Headlines/data/hl2.csv')
hl.head(20)

,headline,city,country,latitude,longitude,countrycode
Unnamed: 0,,,,,,
0,Zika Outbreak Hits Miami,Miami,United States,25.7743,-80.1937,US
1,Could Zika Reach New York City?,New York City,United States,40.7143,-74.006,US
2,First Case of Zika in Miami Beach,Miami Beach,United States,25.7906,-80.13,US
3,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil,-8.05389,-34.8811,BR
4,Dallas man comes down with case of Zika,Dallas,United States,32.7831,-96.8067,US
5,Trinidad confirms first Zika case,Trinidad,Bolivia,-14.8333,-64.9,BO
6,Zika Concerns are Spreading in Houston,Houston,United States,29.7633,-95.3633,US
7,Geneve Scientists Battle to Find Cure,Geneve,,46.2022,6.14569,CH
8,The CDC in Atlanta is Growing Worried,Atlanta,United States,33.749,-84.388,US


# Practices

In [ ]:
## pd element wise methods:
df.add(2)
## element-wise operations:
temp[17] = temp[17].str.strip("%")
temp.replace("%", "")
temp[17].div(100)
pd.to_numeric(temp[17].str.strip("%")).div(100)

## in read_csv or read_json, empty/None will be translated into NaN (not a number).
pd.read_csv("temp_data_01.csv", na_values=['Missing']) # using the na_values parameter to standardize all 'Missing' to NaN

## Other tricks
temp = pd.read_csv("temp_data_01.csv", na_values=['Missing'], header=0, names=range(18), usecols=range(4,18)) # names not columns

## Get rows
calls_revenue[calls_revenue.Territory==3] # can directly use slicing?

## GeoNames
GeoNames database (geonames.org) is for obtaining geographic data. GeoNames contains over eleven million place-names spanning all the countries in the World (continents, countries, and cities, as well US counties and US states). Beyond just place-names, GeoNames also stores valuable information such as latitude and longitude. 

How do we access the GeoNames data? 
1. Download the GeoNames data dump, (download.geonames.org/export/dump/), parse it and then store the output data structure. 
2. GeoNamesCache library: Method are get_continents, get_countries, get_cities, get_countries_by_name, get_cities_by_name, get_us_counties.

### Country Info

In [5]:
continent_dictionary = gc.get_continents()
print(continent_dictionary.keys())

dict_keys(['AF', 'AS', 'EU', 'NA', 'OC', 'SA', 'AN'])


In [7]:
north_america =continent_dictionary['NA']
print(north_america.keys())

dict_keys(['lng', 'geonameId', 'timezone', 'bbox', 'toponymName', 'asciiName', 'astergdem', 'fcl', 'population', 'wikipediaURL', 'adminName5', 'srtm3', 'adminName4', 'adminName3', 'alternateNames', 'cc2', 'adminName2', 'name', 'fclName', 'fcodeName', 'adminName1', 'lat', 'fcode', 'continentCode'])


In [8]:
for name_key in ['name', 'asciiName', 'toponymName']:
    print(north_america[name_key])
    # Various naming schemes. Not very useful.

North America
North America
North America


In [12]:
latitude = float(north_america['lat'])
longitude = float(north_america['lng'])

'''Next Session
fig = plt.figure(figsize=(12, 8))
map_plotter = Basemap(projection='ortho',lat_0=40, lon_0=-95)
map_plotter.scatter([longitude], [latitude], latlon=True, s=200)
map_plotter.drawcoastlines()
map_plotter.drawcountries()
plt.show()
'''


"Next Session\nfig = plt.figure(figsize=(12, 8))\nmap_plotter = Basemap(projection='ortho',lat_0=40, lon_0=-95)\nmap_plotter.scatter([longitude], [latitude], latlon=True, s=200)\nmap_plotter.drawcoastlines()\nmap_plotter.drawcountries()\nplt.show()\n"

In [13]:
countries = gc.get_countries()
num_countries = len(countries)
print(f"GeonamesCache holds data for {num_countries} countries.")
# f-string: add variables

us_data = countries['US']
print("The following data pertains to the United States:")
print(us_data)

GeonamesCache holds data for 252 countries.
The following data pertains to the United States:
{'geonameid': 6252001, 'name': 'United States', 'iso': 'US', 'iso3': 'USA', 'isonumeric': 840, 'fips': 'US', 'continentcode': 'NA', 'capital': 'Washington', 'areakm2': 9629091, 'population': 310232863, 'tld': '.us', 'currencycode': 'USD', 'currencyname': 'Dollar', 'phone': '1', 'postalcoderegex': '^\\d{5}(-\\d{4})?$', 'languages': 'en-US,es-US,haw,fr', 'neighbours': 'CA,MX,CU'}


In [14]:
# The 'neighbours' key maps to a comma-delimited string of country codes that signify all neighboring territories. 
# We can obtain more details about each neighbor by splitting the string and passing the codes into the 'countries' dictionary.
us_neighbors = us_data['neighbours']
for neighbor_code in us_neighbors.split(','):
    print(countries[neighbor_code]['name'])
# At some point, an editor had decided that Cuba is a neighbor the US. Some might disagree with this decision
# Remember that GeoNames is not a golden-standard repository of location information. Some of that data might be imprecise. 
# Please be cautious in your use of GeoNamesCache

Canada
Mexico
Cuba


In [16]:
# get by names
result = gc.get_countries_by_names()['United States']
assert result == countries['US']

### City Info

In [17]:
cities = gc.get_cities()
num_cities = len(cities)
print(f"GeoNamesCache holds data for {num_cities} total cities")
city_id = list(cities.keys())[0]
print(cities[city_id])

GeoNamesCache holds data for 24336 total cities
{'geonameid': 3041563, 'name': 'Andorra la Vella', 'latitude': 42.50779, 'longitude': 1.52109, 'countrycode': 'AD', 'population': 20430, 'timezone': 'Europe/Andorra', 'admin1code': '07'}


In [18]:
## Calculate a country's cities
us_cities = [city for city in cities.values()
             if city['countrycode'] == 'US']
num_us_cities = len(us_cities)
print(f"GeoNamesCache holds data for {num_us_cities} US cities.")

GeoNamesCache holds data for 3249 US cities.


In [19]:
## Approximating US central coordinates
# 1. Average will not take into account the curvature of the Earth. 
# 2. Additionally, the approximation will be inappropriately weighted by city-location. 
#    For instance, a disproportionate number of US cities are located near the American East coast. 
center_lat = np.mean([city['latitude']
                      for city in us_cities])
center_lon = np.mean([city['longitude']
                       for city in us_cities])
'''
fig = plt.figure(figsize=(12, 8))
map_lcc = Basemap(projection='lcc', lon_0=-95, llcrnrlon=-119,
                  llcrnrlat=22, urcrnrlon=-64, urcrnrlat=49, lat_1=33,
                  lat_2=45)
map_lcc.scatter([center_lon], [center_lat], latlon=True, s=200)
map_lcc.drawcoastlines()
map_lcc.drawstates()
plt.show()
'''


"\nfig = plt.figure(figsize=(12, 8))\nmap_lcc = Basemap(projection='lcc', lon_0=-95, llcrnrlon=-119,\n                  llcrnrlat=22, urcrnrlon=-64, urcrnrlat=49, lat_1=33,\n                  lat_2=45)\nmap_lcc.scatter([center_lon], [center_lat], latlon=True, s=200)\nmap_lcc.drawcoastlines()\nmap_lcc.drawstates()\nplt.show()\n"

In [31]:
matched_cities_by_name = gc.get_cities_by_name('Philadelphia')
print(matched_cities_by_name)

AttributeError: 'list' object has no attribute 'keys'

In [22]:
# may return more than one city, because city-names are not always unique.
matched_cities_list = gc.get_cities_by_name('San Francisco')

for i, san_francisco in enumerate(matched_cities_list):
    city_info = list(san_francisco.values())[0]
    country_code = city_info['countrycode']
    country = countries[country_code]['name']
    print(f"The San Francisco at index {i} is located in {country}")

The San Francisco at index 0 is located in Argentina
The San Francisco at index 1 is located in Costa Rica
The San Francisco at index 2 is located in Philippines
The San Francisco at index 3 is located in Philippines
The San Francisco at index 4 is located in El Salvador
The San Francisco at index 5 is located in United States


Its common for multiple cities to share the same name. Choosing among such cities is quite difficult. 
Suppose, for instance, that someone queries a search engine for the "weather in Athens". The search engine must then choose between Athens, Ohio and Athens, Greece. Additional context is required to correctly disambiguate between the locations. Is the user from Ohio? Are they planning a trip to Greece? Without that context, the search engine must guess. 
Usually, the safest guess is the city with the largest population. From a statistical standpoint, the more populous cities are more likely to be referenced in everyday conversation.

In [23]:
best_sf = max(gc.get_cities_by_name('San Francisco'),
              key=lambda x: list(x.values())[0]['population'])
sf_data = list(best_sf.values())[0]
sf_lat = sf_data['latitude']
sf_lon = sf_data['longitude']

'''
fig = plt.figure(figsize=(12, 8))
map_lcc = Basemap(projection='lcc', lon_0=-95, llcrnrlon=-119,
                  llcrnrlat=22, urcrnrlon=-64, urcrnrlat=49, lat_1=33,
                  lat_2=45)
map_lcc.scatter([sf_lon], [sf_lat], latlon=True, s=200)
map_lcc.drawcoastlines()
map_lcc.drawstates()

x, y = map_lcc(sf_lon, sf_lat)
plt.text(x, y, ' San Francisco', fontsize=16)
plt.show()
'''


"\nfig = plt.figure(figsize=(12, 8))\nmap_lcc = Basemap(projection='lcc', lon_0=-95, llcrnrlon=-119,\n                  llcrnrlat=22, urcrnrlon=-64, urcrnrlat=49, lat_1=33,\n                  lat_2=45)\nmap_lcc.scatter([sf_lon], [sf_lat], latlon=True, s=200)\nmap_lcc.drawcoastlines()\nmap_lcc.drawstates()\n\nx, y = map_lcc(sf_lon, sf_lat)\nplt.text(x, y, ' San Francisco', fontsize=16)\nplt.show()\n"

#### Flaws of GeoNames
First of all, the library’s record of cities is far from complete. Certain sparsely populated locations in rural areas (whether the rural United States or rural China) are missing from the stored database records. 
Furthermore, the get_cities_by_name method maps only one version of a city’s name to its geographic data. This poses a problem for cities like New York, which carry more than one commonly referenced name.
The limits of single references become particularly obvious when we examine diacritics (accent marks that designate the proper pronunciation of foreign-sounding words) in city names.

In [24]:
for ny_name in ['New York', 'New York City']:
    if not gc.get_cities_by_name(ny_name):
        print(f"'{ny_name}' is not present in GeoNamesCache database.")
    else:
        print(f"'{ny_name}' is present in GeoNamesCache database.")

'New York' is not present in GeoNamesCache database.
'New York City' is present in GeoNamesCache database.


In [25]:
print(gc.get_cities_by_name(u'Cañon City'))
print(gc.get_cities_by_name(u'Hagåtña'))

[{'5416005': {'geonameid': 5416005, 'name': 'Cañon City', 'latitude': 38.44098, 'longitude': -105.24245, 'countrycode': 'US', 'population': 16400, 'timezone': 'America/Denver', 'admin1code': 'CO'}}]
[{'4044012': {'geonameid': 4044012, 'name': 'Hagåtña', 'latitude': 13.47567, 'longitude': 144.74886, 'countrycode': 'GU', 'population': 1051, 'timezone': 'Pacific/Guam', 'admin1code': 'AN'}}]


Find out how many of the cities stored in GeoNamesCache contain diacritics in their name using the unidecode function from the external Unidecode library. 
These cities are commonly referenced without an accent in published text-data.We can create a mapping to get them.

In [26]:
from unidecode import unidecode
accented_names = [city['name'] for city in gc.get_cities().values()
                  if city['name'] != unidecode(city['name'])]
num_accented_cities = len(accented_names)

print(f"An example accented city name is '{accented_names[0]}'")
print(f"{num_accented_cities} cities have accented names")

An example accented city name is 'Khawr Fakkān'
4904 cities have accented names


In [27]:
# Mapping accents
alternative_names = {unidecode(name): name for name in accented_names}
print(gc.get_cities_by_name(alternative_names['Hagatna']))

[{'4044012': {'geonameid': 4044012, 'name': 'Hagåtña', 'latitude': 13.47567, 'longitude': 144.74886, 'countrycode': 'GU', 'population': 1051, 'timezone': 'Pacific/Guam', 'admin1code': 'AN'}}]


In [28]:
text = u'This sentence matches Hagatna' # Unicode Strings.Python 3 made them redundant as the default string type is Unicode.
for key, value in alternative_names.items():
    if key in text:
        print(gc.get_cities_by_name(value))
        break

[{'4044012': {'geonameid': 4044012, 'name': 'Hagåtña', 'latitude': 13.47567, 'longitude': 144.74886, 'countrycode': 'GU', 'population': 1051, 'timezone': 'Pacific/Guam', 'admin1code': 'AN'}}]
